<center><span style="color:blue"><font size="6">Graph Visualization - pyvis 활용편</font></span></center>  
  
<p><p>RDF 형태의 데이터를 시각화하는 라이브러리는 여러 가지가 있습니다. <p>
이 앞전에는 graph-notebook 이라는 라이브러리를 사용하여 간단하게 그려볼 수 있었는데<p>
보다 상세하고 다이나믹한 그래프를 그리기 위해서 vis.js를 기반으로 사용하고 있는<p>
pyvis 라는 라이브러리를 사용하고자 합니다.<p>
큰 흐름은 시각화를 위한 데이터 추출, 노드와 에지 생성, pyvis로 시각화 하는 단계를 거칩니다. <p>
    
 - 사용하는 라이브러리  
    pyvis 0.1.9  
    (https://pyvis.readthedocs.io/en/latest/) <p>
 - 필수적으로 설치가 되어 있어야 하는 라이브러리
    RDFLib, SPARQLWrapper 

<div align="right">
    작성자 : 허홍수<br>
    e-mail : su4620@gmail.com<br>
    blog : http://joyhong.tistory.com<br>
</div>  

<b>쥬피터 화면을 넒게 쓰기 원하면 아래를 실행</b>

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pyvis 기본 사용법

## 노드, 에지 생성

Network 클래스 인스턴스 생성 후 노드 추가, 에지 추가, 마지막으로 show() 호출  
쥬피터 노트북에서 사용하기 위해  Network(notebook=True) 로 인자값 전달해야 한다. 

In [2]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0)
g.add_node(1)
g.add_edge(0, 1, arrows='to')
g.show("test1.html")

## 그래프 설정 UI  추가

In [3]:
from pyvis.network import Network

g = Network(width='90%', notebook=True)
g.add_node(0)
g.add_node(1)
g.add_edge(0, 1, arrows='to')
g.show_buttons(filter_=['physics'])

g.show("test2.html")


## 노드에 레이블 추가

In [4]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0, label='A')
g.add_node(1, label='B')
g.add_edge(0, 1, arrows='to')
g.show("test3.html")

## 노드에 툴팁 추가

In [5]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0, label='A', title='I am A')
g.add_node(1, label='B', title='I am B')
g.add_node(2, label='C', title='new C')
g.add_edge(0, 1, arrows='to')
g.add_edge(0, 2, arrows='to')
g.show("test4.html")

##  노드 모양, 크기, 색상, 그림자 변경

In [6]:
from pyvis.network import Network

g = Network(notebook=True)
g.add_node(0, label='A', title='I am A', shape='diamond')
g.add_node(1, label='B', title='I am B', shape='box')
g.add_node(2, label='C', title='new C', shape='triangle', size=50)
g.add_node(3, label='D', shape='triangleDown')
g.add_node(4, label='E', shape='square', color='red')
g.add_node(5, label='F', shape='database', color='green', shadow=True)
g.add_edge(0, 1, arrows='to')
g.add_edge(0, 2, arrows='to')
g.show("test5.html")

# SPARQL Endpoint 연동

## 시각화를 위한 기본 구성 세팅

In [7]:
from pyvis.network import Network
import networkx as nx

def getLocalName(node):
    if node.find('/') > 0 and node.find('#') > 0:
        return node[node.rindex('/')+1:] if node.rindex('/') > node.rindex('#') else node[node.rindex('#')+1:]
    elif node.find('/') > 0:
        return node[node.rindex('/')+1:]
    elif node.find('#') > 0:
        return node[node.rindex('/')+1:]
    else:
        return node


def make_short_name(name):
    if len(name) > 20 :
        name = name[0:20]
    return name

def show_graph(results, ignore_property):
    g = Network('800px', '1200px', notebook=True)
    node = {}
    groups = {}
    node_num = 0;
    group_num = 0;
    edge_list = []
    
    for result in results["results"]["bindings"]:
        if not result["p"]["value"] in ignore_property:
            # 존재하지 않는 경우 디폴트로 처리
            if 'slabel' not in result:
                result['slabel'] = {'type': 'literal', 'value': getLocalName(result['s']['value'])}
            if 'olabel' not in result:
                result['olabel'] = {'type': 'literal', 'value': getLocalName(result['o']['value'])}
            if 'stype' not in result:
                result['stype'] = {'type': 'literal', 'value': ''}
            if 'otype' not in result:
                result['otype'] = {'type': 'literal', 'value': ''}

            # 노드
            if result['s']['type'] == 'uri' and not result['s']['value'] in node: 
                node[result['s']['value']] = {'id':node_num}
                node_num += 1

            if result['o']['type'] == 'uri' and not result['o']['value'] in node:
                node[result['o']['value']] = {'id':node_num}
                node_num += 1

            if 'stype'  in result and not result['stype']['value'] in groups:
                groups[result['stype']['value']] = group_num
                group_num += 1
            if 'otype'  in result and not result['otype']['value'] in groups:
                groups[result['otype']['value']] = group_num
                group_num += 1

            node[result['s']['value']].update({'label':result['slabel']['value'], 'type':result['stype']['value']})
            if result['o']['type'] == 'uri':
                node[result['o']['value']].update({'label':result['olabel']['value'], 'type':result['otype']['value']})
                edge_list.append((result['s']['value'], result['o']['value'], result['p']['value']))

    for n in node:
        g.add_node(n, size=25, label=make_short_name(node[n]['label']), title=n, group=groups[node[n]['type']])
    for e in edge_list:
        g.add_edge(e[0], e[1], arrows='to', label=getLocalName(e[2]), title=e[2])
    
    g.set_edge_smooth('dynamic')
    g.show_buttons(filter_=['physics'])
    return g

## SPARQL Endpoint 에 질의하여 결과 받기

아래의 쿼리를 수정하여 사용할 수 있으며, 수정할 때는 반환하는 값이 s, p, o가 있어야 그래프로 그릴 수 있다.

In [8]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib import Graph

sparql = SPARQLWrapper("http://localhost:3030/publicdata/query")

sparql.setQuery("""
    prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT *
    WHERE {
        ?s ?p ?o.
        optional{?s rdfs:label ?slabel .}
        optional{?s a ?stype}
        optional{?o rdfs:label ?olabel .}
        optional{?o a ?otype}
    } LIMIT 500
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

## 그래프로 그리기

In [9]:
g = show_graph(results, [])
g.show("basic.html")

Finish